In [1]:

!pip install transformers datasets

  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
  Using cached numpy-2.1.3-cp310-cp310-macosx_14_0_arm64.whl.metadata (62 kB)
INFO: pip is looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
  Using cached datasets-3.0.2-py3-none-any.whl.metadata (20 kB)
  Using cached datasets-3.0.1-py3-none-any.whl.metadata (20 kB)
  Using cached datasets-3.0.0-py3-none-any.whl.metadata (19 kB)
  Using cached datasets-2.21.0-py3-none-any.whl.metadata (21 kB)
  Using cached datasets-2.20.0-py3-none-any.whl.metadata (19 kB)
  Using cached datasets-2.19.2-py3-none-any.whl.metadata (19 kB)
  Using cached datasets-2.19.1-py3-none-any.whl.metadata (19 kB)
INFO: pip is still looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
  Using cached datasets-2.19.0-py3-none-any.whl.metadata (19 kB)
  Using cached datasets-2.18.0-py3-none-any.w

In [11]:
#!pip install --upgrade pip
!pip install torch
#!pip install torch==2.0.0+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html

  Using cached torch-2.5.1-cp310-none-macosx_11_0_arm64.whl.metadata (28 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.8/63.9 MB 45.9 kB/s eta 0:22:54
ERROR: Exception:
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.10/site-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/opt/homebrew/lib/python3.10/site-packages/pip/_vendor/urllib3/response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "/opt/homebrew/lib/python3.10/site-packages/pip/_vendor/urllib3/response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "/opt/homebrew/lib/python3.10/site-packages/pip/_vendor/cachecontrol/filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
  File "/opt/homebrew/Cellar/python@3.10/3.10.15/Frameworks/

In [7]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer

ModuleNotFoundError: No module named 'torch'

In [ ]:
# Load a pre-trained BART model and tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Ensure model runs on GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
from datasets import load_dataset

# Load CNN/DailyMail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:1%]")  # Use a subset for testing

# Tokenize the dataset
def preprocess_function(examples):
    inputs = examples["article"]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    
    # Tokenize summaries as target labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=128, truncation=True)
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=500,
    remove_unused_columns=False,
    fp16=torch.cuda.is_available(),
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()


In [ ]:
def summarize(text, max_length=50, min_length=10, length_penalty=2.0, num_beams=4):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True).to(device)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=length_penalty, num_beams=num_beams)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Example usage
text = "Your long article or text here..."
print(summarize(text))


In [ ]:
from datasets import load_metric

rouge = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, references = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(references, skip_special_tokens=True)
    
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {key: value.mid.fmeasure for key, value in result.items()}
    return result
